<a href="https://colab.research.google.com/github/DanyalKanzanAhmad/Quarter2-Assignment-05-Exploring-Gemini-2.0-Video-and-Audio-Analysis/blob/main/Assignment_05_Exploring_Gemini_2_0_Video_and_Audio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade --quiet google-genai

In [2]:
!pip install gTTS --quiet

In [3]:
!pip install playsound --quiet

In [18]:
from google.colab import userdata
genai_api:str = userdata.get('Generative_Language_Client')
if(genai_api):
  print("Api Found")
else:
  print("Api Not Found")

Api Found


In [20]:
from google import genai
from google.genai.client import Client

client: Client = genai.Client (
    api_key = genai_api,
                              )
model: str = "gemini-2.0-flash-exp"

In [11]:


# Path to the recorded video
video_file_name = "/content/intro.mp4"


In [28]:
import time

def upload_video(video_file_name):
  video_file = client.files.upload(path=video_file_name)
  while video_file.state == "PROCESSING":
      print('Waiting for video Processing.')
      time.sleep(10)
      video_file = client.files.get(name=video_file.name or "")

  if video_file.state == "FAILED":
    raise ValueError(video_file.state)
  print(f'Video processing Done: ' + (video_file.uri or ""))

  return video_file

my_video = upload_video(video_file_name)



Waiting for video Processing.
Video processing Done: https://generativelanguage.googleapis.com/v1beta/files/45phn2j5ecef


In [33]:
from google.genai.types import Content , Part
prompt = """For each scene in this video,
            genrate captions that describe the scene  """

video = my_video
response = client.models.generate_content(
    model=model,
    contents=[
        Content(
            role="User",
            parts=[
                Part.from_uri(
                    file_uri=video.uri or "",
                    mime_type=video.mime_type or ""),
                Part(text=prompt)  # Include the prompt as a Part within the Parts list
            ]),
       

            ]
     )

print(response.text)

Sure, here are the captions for the video:

[00:00:00-00:00:06] A young man in a blue blazer stands in front of a plain wall. He introduces himself as Jasjot Bhasin from Mumbai, 25 years old and 5'10" tall.
[00:00:06-00:00:10] The young man turns to his left to show his left profile.
[00:00:10-00:00:12] The young man faces the camera.
[00:00:12-00:00:14] The young man turns to his right to show his right profile.
[00:00:14-00:00:17] The young man faces the camera. He states that he speaks English, Hindi and Punjabi.
[00:00:17-00:00:23] The young man talks about his acting experience. He smiles at the end of the video.


In [35]:
from gtts import gTTS
from playsound import playsound
from IPython.display import Audio

tts = gTTS(text=response.text, lang='en')
with open('output.mp3', 'wb') as f:
    for chunk in tts.stream():
        f.write(chunk)
display(Audio('output.mp3', autoplay=True))